In [1]:
%run Weighting.ipynb

Pixiedust database opened successfully


In [2]:
tabel_alternatif = open_tabel("Tabel Alternatif.csv")

tabel_alternatif

,Administrasi Fiskal - UI (IPS),Administrasi Negara - UNPAD (IPS),Administrasi Niaga - UI (IPS),Administrasi Perpajakan - UB (IPS),Agribisnis - UNDIP (IPA),Agroekoteknologi - UB (IPA),Agroteknologi - UNPAD (IPA),Akuntansi - UGM (IPS),Akutansi - UI (IPS),Antropologi - UNAIR (IPS),...,Teknik Sistem Perkapalan - ITS (IPA),Teknobiomedik - UNAIR (IPA),Teknologi Bioproses - UI (IPA),Teknologi Hasil Perikanan - UB (IPA),Teknologi Industri Hasil Perikanan - UNAIR (IPA),Teknologi Industri Pertanian - UGM (IPA),Teknologi Pangan - UB (IPA),Televisi & Film - UNPAD (IPS),Tip - UGM (IPA),Tv & Film - UNPAD (IPS)
Matematika,84.000000,88.500000,85.465625,92.000000,86.250000,81.852143,82.500000,91.625000,44.693125,84.576004,...,90.000000,86.154669,85.465625,82.000000,85.000000,86.506173,82.408571,85.000000,87.000000,85.000000
Inggris,84.000000,80.500000,83.888667,86.000000,85.500000,84.729722,82.750000,90.875000,54.255467,85.307966,...,87.000000,86.153983,83.888667,91.000000,84.000000,86.859259,83.918889,85.500000,94.000000,85.000000
Indonesia,84.000000,89.500000,85.246032,89.000000,87.833333,85.837917,84.000000,89.875000,85.000000,85.970128,...,88.000000,86.651731,85.246032,89.000000,86.000000,86.982716,84.351667,86.500000,80.000000,84.000000
Fisika,82.000000,84.584430,84.493016,81.956615,83.360000,81.739154,80.500000,89.437576,87.000000,84.646720,...,88.500000,85.048360,84.493016,80.000000,83.000000,85.562309,81.956615,84.584430,86.000000,84.584430
Kimia,81.000000,84.592411,85.297674,82.397782,85.100000,82.349446,81.500000,88.716106,74.300000,85.035214,...,89.500000,85.286546,85.297674,79.000000,81.000000,86.895425,82.397782,84.592411,87.000000,84.592411
Biologi,83.000000,83.879942,84.182558,82.827536,86.172727,83.706884,83.250000,89.695798,83.000000,86.206897,...,92.000000,86.364812,84.182558,83.000000,86.000000,87.039216,82.827536,83.879942,86.000000,83.879942
Ekonomi,86.145714,89.000000,86.145714,91.242143,87.271429,83.726429,82.062045,92.250000,86.145714,85.211111,...,87.839744,85.654365,86.145714,88.726429,85.211111,87.864219,84.210714,86.000000,89.837895,86.000000
Geografi,85.512727,87.500000,85.512727,91.000000,87.500000,84.627747,81.926031,88.959803,85.512727,84.793706,...,84.333333,85.363520,85.512727,84.808608,84.793706,88.470835,84.808608,86.500000,88.094322,87.000000
Sosiologi,86.930877,85.500000,86.930877,91.000000,87.000000,85.702500,81.331846,91.872475,86.930877,85.695971,...,84.666667,85.531319,86.930877,84.850000,85.695971,91.380471,84.850000,85.500000,90.363636,85.000000


In [3]:
nilai_jams = [70, 70, 70, 70, 70, 70, 70, 70, 70,]

In [4]:
mean = get_mean(tabel_alternatif)
std = get_std(tabel_alternatif)
params = get_params(mean, nilai_jams, std)

params

[-2.749822364309754,
 -3.638069364917934,
 -4.8201766621884445,
 -3.7731279264212465,
 -3.605954356872745,
 -4.615593197475401,
 -6.099829348527156,
 -6.445788711418059,
 -5.758986068783327]

In [5]:
tabel_alternatif = add_parameter(params, tabel_alternatif)

tabel_alternatif

,Administrasi Fiskal - UI (IPS),Administrasi Negara - UNPAD (IPS),Administrasi Niaga - UI (IPS),Administrasi Perpajakan - UB (IPS),Agribisnis - UNDIP (IPA),Agroekoteknologi - UB (IPA),Agroteknologi - UNPAD (IPA),Akuntansi - UGM (IPS),Akutansi - UI (IPS),Antropologi - UNAIR (IPS),...,Teknobiomedik - UNAIR (IPA),Teknologi Bioproses - UI (IPA),Teknologi Hasil Perikanan - UB (IPA),Teknologi Industri Hasil Perikanan - UNAIR (IPA),Teknologi Industri Pertanian - UGM (IPA),Teknologi Pangan - UB (IPA),Televisi & Film - UNPAD (IPS),Tip - UGM (IPA),Tv & Film - UNPAD (IPS),Parameter
Matematika,84.000000,88.500000,85.465625,92.000000,86.250000,81.852143,82.500000,91.625000,44.693125,84.576004,...,86.154669,85.465625,82.000000,85.000000,86.506173,82.408571,85.000000,87.000000,85.000000,-2.749822
Inggris,84.000000,80.500000,83.888667,86.000000,85.500000,84.729722,82.750000,90.875000,54.255467,85.307966,...,86.153983,83.888667,91.000000,84.000000,86.859259,83.918889,85.500000,94.000000,85.000000,-3.638069
Indonesia,84.000000,89.500000,85.246032,89.000000,87.833333,85.837917,84.000000,89.875000,85.000000,85.970128,...,86.651731,85.246032,89.000000,86.000000,86.982716,84.351667,86.500000,80.000000,84.000000,-4.820177
Fisika,82.000000,84.584430,84.493016,81.956615,83.360000,81.739154,80.500000,89.437576,87.000000,84.646720,...,85.048360,84.493016,80.000000,83.000000,85.562309,81.956615,84.584430,86.000000,84.584430,-3.773128
Kimia,81.000000,84.592411,85.297674,82.397782,85.100000,82.349446,81.500000,88.716106,74.300000,85.035214,...,85.286546,85.297674,79.000000,81.000000,86.895425,82.397782,84.592411,87.000000,84.592411,-3.605954
Biologi,83.000000,83.879942,84.182558,82.827536,86.172727,83.706884,83.250000,89.695798,83.000000,86.206897,...,86.364812,84.182558,83.000000,86.000000,87.039216,82.827536,83.879942,86.000000,83.879942,-4.615593
Ekonomi,86.145714,89.000000,86.145714,91.242143,87.271429,83.726429,82.062045,92.250000,86.145714,85.211111,...,85.654365,86.145714,88.726429,85.211111,87.864219,84.210714,86.000000,89.837895,86.000000,-6.099829
Geografi,85.512727,87.500000,85.512727,91.000000,87.500000,84.627747,81.926031,88.959803,85.512727,84.793706,...,85.363520,85.512727,84.808608,84.793706,88.470835,84.808608,86.500000,88.094322,87.000000,-6.445789
Sosiologi,86.930877,85.500000,86.930877,91.000000,87.000000,85.702500,81.331846,91.872475,86.930877,85.695971,...,85.531319,86.930877,84.850000,85.695971,91.380471,84.850000,85.500000,90.363636,85.000000,-5.758986


In [6]:
pilihan_jurusan = get_jurusan(tabel_alternatif)

for pilihan in pilihan_jurusan:
    if "(ipa)" in pilihan.casefold().split():
        print(pilihan)

Agribisnis - UNDIP (IPA)
Agroekoteknologi - UB (IPA)
Agroteknologi - UNPAD (IPA)
Apoteker - UNAIR (IPA)
Arsitektur - ITS (IPA)
Astronomi - ITB (IPA)
Biologi - ITS (IPA)
Budidaya Perairan - UNAIR (IPA)
Fakultas Ilmu Dan Teknologi Kebumian - ITB (IPA)
Fakultas Kedokteran Gigi - UNAIR (IPA)
Fakultas Teknik Sipil & Lingkungan - ITB (IPA)
Farmasi - UI (IPA)
Fisika - ITS (IPA)
Fitb - ITB (IPA)
Fkg - UGM (IPA)
Fmipa - ITB (IPA)
Fmipa Fisika - UB (IPA)
Fmipa- Ilmu Kimia - UNPAD (IPA)
Fti - ITB (IPA)
Geofisika - UGM (IPA)
Geofisika  - UI (IPA)
Gizi - UGM (IPA)
Ilmu Gizi - UI (IPA)
Ilmu Hub. Internasional - UNPAD (IPA)
Ilmu Kelautan - UNDIP (IPA)
Ilmu Keperawatan - UI (IPA)
Ilmu Keperawatan  - UNPAD  (IPA)
Ilmu Kesehatan Masyarakat - UI (IPA)
Ilmu Komputer - UGM (IPA)
Ilmu Peternakan - UNPAD (IPA)
Itb - Sithr - ITB (IPA)
Kedokteran - UI (IPA)
Kedokteran  - UNAIR (IPA)
Kedokteran Gigi - UGM (IPA)
Kedokteran Hewan - UNAIR (IPA)
Kehutanan - UGM (IPA)
Kesehatan Masyarakat - UI (IPA)
Kesehatan Masyar

In [7]:
fp_gaussian = fungsi_preferensi_gaussian(tabel_alternatif)

fp_gaussian

,Administrasi Fiskal - UI (IPS)&Administrasi Negara - UNPAD (IPS),Administrasi Fiskal - UI (IPS)&Administrasi Niaga - UI (IPS),Administrasi Fiskal - UI (IPS)&Administrasi Perpajakan - UB (IPS),Administrasi Fiskal - UI (IPS)&Agribisnis - UNDIP (IPA),Administrasi Fiskal - UI (IPS)&Agroekoteknologi - UB (IPA),Administrasi Fiskal - UI (IPS)&Agroteknologi - UNPAD (IPA),Administrasi Fiskal - UI (IPS)&Akuntansi - UGM (IPS),Administrasi Fiskal - UI (IPS)&Akutansi - UI (IPS),Administrasi Fiskal - UI (IPS)&Antropologi - UNAIR (IPS),Administrasi Fiskal - UI (IPS)&Antropologi Sosial - UB (IPS),...,Tv & Film - UNPAD (IPS)&Teknik Sipil - ITB (IPA),Tv & Film - UNPAD (IPS)&Teknik Sistem Perkapalan - ITS (IPA),Tv & Film - UNPAD (IPS)&Teknobiomedik - UNAIR (IPA),Tv & Film - UNPAD (IPS)&Teknologi Bioproses - UI (IPA),Tv & Film - UNPAD (IPS)&Teknologi Hasil Perikanan - UB (IPA),Tv & Film - UNPAD (IPS)&Teknologi Industri Hasil Perikanan - UNAIR (IPA),Tv & Film - UNPAD (IPS)&Teknologi Industri Pertanian - UGM (IPA),Tv & Film - UNPAD (IPS)&Teknologi Pangan - UB (IPA),Tv & Film - UNPAD (IPS)&Televisi & Film - UNPAD (IPS),Tv & Film - UNPAD (IPS)&Tip - UGM (IPA)
Matematika,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Inggris,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Indonesia,1,1,1,1,1,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Fisika,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Kimia,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Biologi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ekonomi,1,0,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,1,0,1
Geografi,1,0,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
Sosiologi,1,0,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1


In [8]:
index_preferensi_global = idx_pref_global(fp_gaussian, weight)

index_preferensi_global

,Administrasi Fiskal - UI (IPS)&Administrasi Negara - UNPAD (IPS),Administrasi Fiskal - UI (IPS)&Administrasi Niaga - UI (IPS),Administrasi Fiskal - UI (IPS)&Administrasi Perpajakan - UB (IPS),Administrasi Fiskal - UI (IPS)&Agribisnis - UNDIP (IPA),Administrasi Fiskal - UI (IPS)&Agroekoteknologi - UB (IPA),Administrasi Fiskal - UI (IPS)&Agroteknologi - UNPAD (IPA),Administrasi Fiskal - UI (IPS)&Akuntansi - UGM (IPS),Administrasi Fiskal - UI (IPS)&Akutansi - UI (IPS),Administrasi Fiskal - UI (IPS)&Antropologi - UNAIR (IPS),Administrasi Fiskal - UI (IPS)&Antropologi Sosial - UB (IPS),...,Tv & Film - UNPAD (IPS)&Teknik Sipil - ITB (IPA),Tv & Film - UNPAD (IPS)&Teknik Sistem Perkapalan - ITS (IPA),Tv & Film - UNPAD (IPS)&Teknobiomedik - UNAIR (IPA),Tv & Film - UNPAD (IPS)&Teknologi Bioproses - UI (IPA),Tv & Film - UNPAD (IPS)&Teknologi Hasil Perikanan - UB (IPA),Tv & Film - UNPAD (IPS)&Teknologi Industri Hasil Perikanan - UNAIR (IPA),Tv & Film - UNPAD (IPS)&Teknologi Industri Pertanian - UGM (IPA),Tv & Film - UNPAD (IPS)&Teknologi Pangan - UB (IPA),Tv & Film - UNPAD (IPS)&Televisi & Film - UNPAD (IPS),Tv & Film - UNPAD (IPS)&Tip - UGM (IPA)
Index Preferensi Global,0.526919,0.210768,0.526919,0.526919,0.526919,0.421535,0.526919,0.210768,0.526919,0.526919,...,0.526919,0.526919,0.526919,0.526919,0.526919,0.526919,0.526919,0.526919,0.421535,0.526919


In [9]:
e_flow = entering_flow(pilihan_jurusan, index_preferensi_global)

e_flow

{'Administrasi Fiskal - UI (IPS)': 0.5812445027289649,
 'Administrasi Negara - UNPAD (IPS)': 0.5947796330027391,
 'Administrasi Niaga - UI (IPS)': 0.576991086372726,
 'Administrasi Perpajakan - UB (IPS)': 0.5832069965563498,
 'Agribisnis - UNDIP (IPA)': 0.5920928395997763,
 'Agroekoteknologi - UB (IPA)': 0.6044740783375436,
 'Agroteknologi - UNPAD (IPA)': 0.5998247911071959,
 'Akuntansi - UGM (IPS)': 0.6056279698735506,
 'Akutansi - UI (IPS)': 0.5901303457723915,
 'Antropologi - UNAIR (IPS)': 0.6033033262583767,
 'Antropologi Sosial - UB (IPS)': 0.6025284450533187,
 'Apoteker - UNAIR (IPA)': 0.5940216123408413,
 'Arkeologi - UI (IPS)': 0.6056279698735506,
 'Arsitektur - ITS (IPA)': 0.5824152548081319,
 'Astronomi - ITB (IPA)': 0.5785577093260021,
 'Biologi - ITS (IPA)': 0.582432115351292,
 'Budidaya Perairan - UNAIR (IPA)': 0.6036991971324857,
 'Desain Interior - ITS (IPS)': 0.6048530886684926,
 'Ekonomi Islam - UB (IPS)': 0.5839481566750876,
 'Ekonomi Islam  - UNPAD (IPS)': 0.59168010

In [10]:
l_flow = leaving_flow(pilihan_jurusan, index_preferensi_global)

l_flow

{'Administrasi Fiskal - UI (IPS)': 0.49902349605732405,
 'Administrasi Negara - UNPAD (IPS)': 0.5222699322090625,
 'Administrasi Niaga - UI (IPS)': 0.5013481396724977,
 'Administrasi Perpajakan - UB (IPS)': 0.5145211201584832,
 'Agribisnis - UNDIP (IPA)': 0.6730169920087016,
 'Agroekoteknologi - UB (IPA)': 0.683848468336353,
 'Agroteknologi - UNPAD (IPA)': 0.6776494186958893,
 'Akuntansi - UGM (IPS)': 0.5269192194394101,
 'Akutansi - UI (IPS)': 0.5052225456977876,
 'Antropologi - UNAIR (IPS)': 0.5152960013635408,
 'Antropologi Sosial - UB (IPS)': 0.5230448134141205,
 'Apoteker - UNAIR (IPA)': 0.6733960023396507,
 'Arkeologi - UI (IPS)': 0.5176206449787147,
 'Arsitektur - ITS (IPA)': 0.6602398823968254,
 'Astronomi - ITB (IPA)': 0.6560033265837467,
 'Biologi - ITS (IPA)': 0.6598777326090364,
 'Budidaya Perairan - UNAIR (IPA)': 0.6830735871312952,
 'Desain Interior - ITS (IPS)': 0.517620644978715,
 'Ekonomi Islam - UB (IPS)': 0.4912746840067442,
 'Ekonomi Islam  - UNPAD (IPS)': 0.5168457

In [11]:
n_flow = net_flow(pilihan_jurusan, e_flow, l_flow)

n_flow

{'Administrasi Fiskal - UI (IPS)': -0.08222100667164084,
 'Administrasi Negara - UNPAD (IPS)': -0.07250970079367658,
 'Administrasi Niaga - UI (IPS)': -0.07564294670022831,
 'Administrasi Perpajakan - UB (IPS)': -0.06868587639786661,
 'Agribisnis - UNDIP (IPA)': 0.08092415240892525,
 'Agroekoteknologi - UB (IPA)': 0.07937438999880941,
 'Agroteknologi - UNPAD (IPA)': 0.07782462758869346,
 'Akuntansi - UGM (IPS)': -0.0787087504341405,
 'Akutansi - UI (IPS)': -0.08490780007460397,
 'Antropologi - UNAIR (IPS)': -0.08800732489483587,
 'Antropologi Sosial - UB (IPS)': -0.07948363163919825,
 'Apoteker - UNAIR (IPA)': 0.07937438999880941,
 'Arkeologi - UI (IPS)': -0.08800732489483587,
 'Arsitektur - ITS (IPA)': 0.07782462758869346,
 'Astronomi - ITB (IPA)': 0.07744561725774457,
 'Biologi - ITS (IPA)': 0.07744561725774435,
 'Budidaya Perairan - UNAIR (IPA)': 0.07937438999880952,
 'Desain Interior - ITS (IPS)': -0.08723244368977767,
 'Ekonomi Islam - UB (IPS)': -0.09267347266834342,
 'Ekonomi Is

In [12]:
hasil_outrank = sorted(n_flow.items(), key = lambda kv:(kv[1], kv[0]), reverse = True)

tab = dict()

rank = list()
jurusan = list()
net = list()

for i in range(len(hasil_outrank)):
    rank.append(i+1)
    jurusan.append(hasil_outrank[i][0])
    net.append(hasil_outrank[i][1])
    
tab["Jurusan"] = jurusan
tab["Net Flow"] = net

tabel_hasil = pd.DataFrame(tab, index=rank)

tabel_hasil

,Jurusan,Net Flow
1,Pendidikan Dokter - UB (IPA),0.087519
2,Mipa Kimia - UB (IPA),0.086744
3,Teknik Lingkungan - UB (IPA),0.085969
4,Matematika - UI (IPA),0.085573
5,Fmipa Fisika - UB (IPA),0.085194
...,...,...
133,Sosiologi - UB (IPS),-0.092673
134,Ilmu Administrasi Publik - UB (IPS),-0.092673
135,Ekonomi Pembangunan - UB (IPS),-0.092673
136,Ekonomi Islam - UB (IPS),-0.092673
